In [1]:
%run startup.py

In [2]:
%%javascript
$.getScript('./assets/js/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# A Decision Tree of Observable Operators

## Part 2: Combining Observables

> source: http://reactivex.io/documentation/operators.html#tree.  
> (transcribed to RxPY 1.5.7, Py2.7 / 2016-12, Gunther Klessinger, [axiros](http://www.axiros.com))  

**This tree can help you find the ReactiveX Observable operator you’re looking for.**  
See [Part 1](./A Decision Tree of Observable Operators. Part I - Creation.ipynb) for Usage and Output Instructions.  
We also require acquaintance with the [marble diagrams](./Marble Diagrams.ipynb) feature of RxPy.

[This](http://www.introtorx.com/Content/v1.0.10621.0/12_CombiningSequences.html) is a helpful accompanying read.  
<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>



# I want to create an Observable by **combining** other Observables

## ... and emitting all of the items from all of the Observables in whatever order they are received: **[merge / merge_all](http://reactivex.io/documentation/operators/merge.html) **

In [7]:
from reactivex import operators as ops
reset_start_time(ops.merge)
l = []
def excepting_f(obs):
    for i in range(10):
        l.append(1)
        obs.on_next(1 / (3 - len(l)))
stream1 = reactivex.from_(('a', 'b', 'c'))
stream2 = reactivex.create(excepting_f)
# merged stream stops in any case at first exception!
# No guarantee of order of those immediately created streams !
d = subs(stream1.pipe(ops.merge(stream2)))
l = []
d = subs(ops.merge(new_thread_scheduler, [stream1, stream2]))



========== merge ==========

module reactivex.operators
def merge(
    *sources: Observable[Any], max_concurrent: Optional[int] = None
):
    Merges an observable sequence of observable sequences into an
    observable sequence, limiting the number of concurrent
    subscriptions to inner sequences. Or merges two observable
    sequences into a single observable sequence.

    .. marble::
        :alt: merge

        ---1---2---3---4-|
        -a---b---c---d--|
        [     merge()      ]
        -a-1-b-2-c-3-d-4-|

    Examples:
        >>> op = merge(max_concurrent=1)
        >>> op = merge(other_source)

    Args:
        max_concurrent: [Optional] Maximum number of inner observable
            sequences being subscribed to concurrently or the second
            observable sequence.

    Returns:
        An operator function that takes an observable source and
        returns the observable sequence that merges the elements of the
        inner sequences.
------------------------

NameError: name 'new_thread_scheduler' is not defined

In [20]:
rst(ops.merge_all, title='merge_all')
meta = reactivex.from_((1, 2, 3)).pipe(ops.repeat(3))
# no guarantee of order, immediatelly created:
d = subs(meta.pipe(ops.merge_all()))
# Introducing delta ts:
d = subs(reactivex.timer(10, 10)\
                  .pipe(
                  ops.take(3),
                ops.repeat(3),\
                  ops.merge_all()),
         name='streams with time delays between events')




========== merge_all ==========

module reactivex.operators
def merge_all():
    The merge_all operator.

    Merges an observable sequence of observable sequences into an
    observable sequence.

    .. marble::
        :alt: merge_all

        ---1---2---3---4-|
        -a---b---c---d--|
        [   merge_all()    ]
        -a-1-b-2-c-3-d-4-|

    Returns:
        A partially applied operator function that takes an observable
        source and returns the observable sequence that merges the
        elements of the inner sequences.
--------------------------------------------------------------------------------

   1.2     M New subscription (52102) on stream 302153710
   1.5     M [err ]    0.3: 'int' object has no attribute 'subscribe' -> 52102

   1.8     M New subscription (streams with time delays between events) on stream 302265579


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/nix/store/cgds5z15zf6y3pfyb63blvfb0ymz8pvp-python3-3.9.12/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/nix/store/cgds5z15zf6y3pfyb63blvfb0ymz8pvp-python3-3.9.12/lib/python3.9/threading.py", line 1299, in run
    self.function(*self.args, **self.kwargs)
  File "/Users/joel/Desktop/projects/RxPY/.en/lib/python3.9/site-packages/reactivex/scheduler/timeoutscheduler.py", line 89, in interval
    sad.disposable = self.invoke_action(action, state)
  File "/Users/joel/Desktop/projects/RxPY/.en/lib/python3.9/site-packages/reactivex/scheduler/scheduler.py", line 105, in invoke_action
    ret = action(self, state)
  File "/Users/joel/Desktop/projects/RxPY/.en/lib/python3.9/site-packages/reactivex/scheduler/periodicscheduler.py", line 49, in periodic
    state = action(state)
  File "/Users/joel/Desktop/projects/RxPY/.en/lib/python3.9/site-packages/reactivex/observable/timer.py", line 10

## ... and emitting all of the items from all of the Observables, one Observable at a time: **[concat](http://reactivex.io/documentation/operators/concat.html) **

In [29]:
rst(ops.concat)
s1 = reactivex.from_((1, 2))
s2 = reactivex.from_((3, 4))
# while normal subscriptions work as expected...
d1, d2 = subs(s1), subs(s2)
# ... another one can have the order reversed
d = subs(s2.pipe(ops.concat(s1)))



========== concat ==========

module reactivex.operators
def concat(*sources: Observable[_T]):
    Concatenates all the observable sequences.

    .. marble::
        :alt: concat

        ---1--2--3--|
        --6--8--|
        [     concat()     ]
        ---1--2--3----6--8-|

    Examples:
        >>> op = concat(xs, ys, zs)

    Returns:
        An operator function that takes one or more observable sources and
        returns an observable sequence that contains the elements of
        each given sequence, in sequential order.
--------------------------------------------------------------------------------

   1.6     M New subscription (38918) on stream 287399837
   1.8     M [next]    0.2: 1 -> 38918
   1.8     M [next]    0.2: 2 -> 38918
   1.8     M [cmpl]    0.2: fin -> 38918

   1.9     M New subscription (38912) on stream 287399741
   2.0     M [next]    0.1: 3 -> 38912
   2.0     M [next]    0.1: 4 -> 38912
   2.0     M [cmpl]    0.1: fin -> 38912

   2.1     M New subsc

In [32]:
rst()
# See the marbles notebook:
s1 = reactivex.from_marbles('1--2---3|').to_blocking()
s2 = reactivex.from_marbles('--a-b-c|' ).to_blocking()
d = (subs(s1, name='A'),
     subs(s2, name='B'))
rst(title="Concatenating in reverse order", sleep=1)
d = subs(O.concat([s2, s1]), name='C')



========== True ==========



AttributeError: module 'reactivex.operators' has no attribute 'to_blocking'

## ... by combining the items from two or more Observables sequentially to come up with new items to emit

### ... whenever *each* of the Observables has emitted a new item **[zip / zip_list](http://reactivex.io/documentation/operators/zip.html)**

In [176]:
rst(O.zip)
s1 = O.range(0, 5)
d = subs(O.zip(s1, s1.skip(1), s1.skip(2), lambda s1, s2, s3: '%s : %s : %s' % (s1, s2, s3)))

function zip of module rx.linq.observable.zip:
Merges the specified observable sequences into one observable
    sequence by using the mapper function whenever all of the observable
    sequences have produced an element at a corresponding index.

    The last element in the arguments must be a function to invoke for each
    series of elements at corresponding indexes in the sources.

    Arguments:
    args -- Observable sources.

    Returns an observable {Observable} sequence containing the result of
    combining elements of the sources using the specified result mapper
    function.
    
--------------------------------------------------------------------------------

   0.8     M New subscription on stream 276463149
   3.0     M [next]    2.2: 0 : 1 : 2
   3.5     M [next]    2.6: 1 : 2 : 3
   3.9     M [next]    3.0: 2 : 3 : 4
   4.3     M [cmpl]    3.4: fin


In [179]:
rst(O.zip_list) # alias: zip_array
s1 = O.range(0, 5)
d = subs(O.zip_list(s1, s1.skip(1), s1.skip(2)))

function zip_list of module rx.linq.observable.ziparray:
Merge the specified observable sequences into one observable
    sequence by emitting a list with the elements of the observable
    sequences at corresponding indexes.

    Keyword arguments:
    :param Observable cls: Class
    :param Tuple args: Observable sources.

    :return: Returns an observable sequence containing lists of
    elements at corresponding indexes.
    :rtype: Observable
    
--------------------------------------------------------------------------------

   0.9     M New subscription on stream 276480037
   2.9     M [next]    1.8: [0, 1, 2]
   3.3     M [next]    2.2: [1, 2, 3]
   3.9     M [next]    2.8: [2, 3, 4]
   4.4     M [cmpl]    3.3: fin


### ... whenever *any* of the Observables has emitted a new item **[combine_latest](http://reactivex.io/documentation/operators/combinelatest.html)**

In [17]:
rst(O.combine_latest, title='combine_latest')
s1 = O.interval(100).map(lambda i: 'First : %s' % i)
s2 = O.interval(150).map(lambda i: 'Second: %s' % i)
# the start is interesting, both must have emitted, so it starts at 150ms with 0/0:
d = subs(s1.combine_latest(s2, lambda s1, s2: '%s, %s' % (s1, s2)).take(6))

rst(title='For comparison: merge', sleep=1)
d = subs(s1.merge(s2).take(6))




========== combine_latest ==========

function combine_latest of module rx.linq.observable.combinelatest:
Merges the specified observable sequences into one observable
    sequence by using the mapper function whenever any of the
    observable sequences produces an element.

    1 - obs = Observable.combine_latest(obs1, obs2, obs3,
                                       lambda o1, o2, o3: o1 + o2 + o3)
    2 - obs = Observable.combine_latest([obs1, obs2, obs3],
                                        lambda o1, o2, o3: o1 + o2 + o3)

    Returns an observable sequence containing the result of combining
    elements of the sources using the specified result mapper
    function.
    
--------------------------------------------------------------------------------

   1.0     M New subscription on stream 276805405
   3.0     M main thread sleeping 1s
 157.1  T195 [next]  156.1: First : 0, Second: 0
 211.6  T196 [next]  210.6: First : 1, Second: 0
 312.5  T197 [next]  311.6: First : 1, 

### ... whenever *the first* of the Observables has emitted a new item **[with_latest_from](http://reactivex.io/documentation/operators/combinelatest.html)**

In [16]:
rst(O.with_latest_from, title='with_latest_from')
s1 = O.interval(140).map(lambda i: 'First : %s' % i)
s2 = O.interval(50) .map(lambda i: 'Second: %s' % i)
d = subs(s1.with_latest_from(s2, lambda s1, s2: '%s, %s' % (s1, s2)).take(6))




========== with_latest_from ==========

function with_latest_from of module rx.linq.observable.withlatestfrom:
Merges the specified observable sequences into one observable sequence
    by using the mapper function only when the first observable sequence
    produces an element. The observables can be passed either as seperate
    arguments or as a list.

    1 - obs = Observable.with_latest_from(obs1, obs2, obs3,
                                       lambda o1, o2, o3: o1 + o2 + o3)
    2 - obs = Observable.with_latest_from([obs1, obs2, obs3],
                                        lambda o1, o2, o3: o1 + o2 + o3)

    Returns an observable sequence containing the result of combining
    elements of the sources using the specified result mapper function.
    
--------------------------------------------------------------------------------

   1.0     M New subscription on stream 276799777
 147.0  T170 [next]  146.0: First : 0, Second: 1
 292.0  T174 [next]  290.9: First : 1, Secon

### ... whenever an item is emitted by one Observable in a window defined by an item emitted by another **[join](http://reactivex.io/documentation/operators/join.html)**
The join operator takes four parameters:

1. the second Observable to combine with the source Observable
1. a function that accepts an item from the source Observable and returns an Observable whose lifespan governs the duration during which that item will combine with items from the second Observable
1. a function that accepts an item from the second Observable and returns an Observable whose lifespan governs the duration during which that item will combine with items from the first Observable
1. a function that accepts an item from the first Observable and an item from the second Observable and returns an item to be emitted by the Observable returned from join


In [23]:
rst(O.join)
# this one is pretty timing critical and output seems swallowed with 2 threads (over)writing.
# better try this with timer(0) on the console. Also the scheduler of the timers is critical,
# try other O.timer schedulers...
xs = O.interval(100).map(lambda i: 'First : %s' % i)
ys = O.interval(101).map(lambda i: 'Second: %s' % i)
d = subs(xs.join(ys, lambda _: O.timer(10), lambda _: O.timer(0), lambda x, y: '%s %s' % (x, y)).take(5))

function join of module rx.linq.observable.join:
Correlates the elements of two sequences based on overlapping
    durations.

    Keyword arguments:
    right -- The right observable sequence to join elements for.
    left_duration_mapper -- A function to select the duration (expressed
        as an observable sequence) of each element of the left observable
        sequence, used to determine overlap.
    right_duration_mapper -- A function to select the duration (expressed
        as an observable sequence) of each element of the right observable
        sequence, used to determine overlap.
    result_mapper -- A function invoked to compute a result element for
        any two overlapping elements of the left and right observable
        sequences. The parameters passed to the function correspond with
        the elements from the left and right source sequences for which
        overlap occurs.

    Return an observable sequence that contains result elements computed
    from sourc

### ... or, alternatively, **[group_join](http://reactivex.io/documentation/operators/join.html)**

The groupJoin operator takes four parameters:

1. the second Observable to combine with the source Observable
1. a function that accepts an item from the source Observable and returns an Observable whose lifespan governs the duration during which that item will combine with items from the second Observable
1. a function that accepts an item from the second Observable and returns an Observable whose lifespan governs the duration during which that item will combine with items from the first Observable
1. a function that accepts an item from the first Observable and an Observable that emits items from the second Observable and returns an item to be emitted by the Observable returned from groupJoin


In [27]:
rst(O.group_join, title='group_join')
xs = O.interval(100).map(lambda i: 'First : %s' % i)
ys = O.interval(100).map(lambda i: 'Second: %s' % i)
d = subs(xs.group_join(ys,
                       lambda _: O.timer(0),
                       lambda _: O.timer(0),
                       lambda x, yy: yy.select(lambda y: '%s %s' % (x, y))).merge_all().take(5))



========== group_join ==========

function group_join of module rx.linq.observable.groupjoin:
Correlates the elements of two sequences based on overlapping
    durations, and groups the results.

    Keyword arguments:
    right -- The right observable sequence to join elements for.
    left_duration_mapper -- A function to select the duration (expressed
        as an observable sequence) of each element of the left observable
        sequence, used to determine overlap.
    right_duration_mapper -- A function to select the duration (expressed
        as an observable sequence) of each element of the right observable
        sequence, used to determine overlap.
    result_mapper -- A function invoked to compute a result element for
        any element of the left sequence with overlapping elements from the
        right observable sequence. The first parameter passed to the
        function is an element of the left sequence. The second parameter
        passed to the function is an 

### ... by means of Pattern and Plan intermediaries And/Then/When, **[and / then / when](http://reactivex.io/documentation/operators/and-then-when.html)**
The combination of the And, Then, and When operators behave much like the Zip operator, but they do so by means of intermediate data structures. And accepts two or more Observables and combines the emissions from each, one set at a time, into Pattern objects. Then operates on such Pattern objects, transforming them in a Plan. When in turn transforms these various Plan objects into emissions from an Observable.

[details](http://www.introtorx.com/content/v1.0.10621.0/12_CombiningSequences.html#AndThenWhen) 
The And/Then/When trio has more overloads that enable you to group an even greater number of sequences. They also allow you to provide more than one 'plan' (the output of the Then method). This gives you the Merge feature but on the collection of 'plans'. I would suggest playing around with them if this functionality is of interest to you. The verbosity of enumerating all of the combinations of these methods would be of low value. You will get far more value out of using them and discovering for yourself.

As we delve deeper into the depths of what the Rx libraries provide us, we can see more practical usages for it. Composing sequences with Rx allows us to easily make sense of the multiple data sources a problem domain is exposed to. We can concatenate values or sequences together sequentially with StartWith, Concat and Repeat. We can process multiple sequences concurrently with Merge, or process a single sequence at a time with Amb and Switch. Pairing values with CombineLatest, Zip and the And/Then/When operators can simplify otherwise fiddly operations like our drag-and-drop examples and monitoring system status.




In [59]:
rst()
# see the similarity to zip. 
ts = time.time()
def _dt():
    # giving us info when an element was created:
    return 'from time: %.2f' % (time.time() - ts)
one = O.interval(1000) .map(lambda i: 'Seconds : %s %s' % (i, _dt())).take(5)
two = O.interval(500)  .map(lambda i: 'HalfSecs: %s %s' % (i, _dt())).take(5)
three = O.interval(100).map(lambda i: '10thS   : %s %s' % (i, _dt())).take(5)

z = O.when(
   one          \
    .and_(two)  \
    .and_(three)\
    .then_do(lambda a, b, c: '\n'.join(('', '', a, b, c))))

# from the output you see that the result stream consists of elements built at each interval
# (which is in the past for 'two' and 'three'),
# buffered until the 1 second sequence 'one' advances a step. 
d = subs(z)



   1.1     M New subscription on stream 275998165
1002.9 T1468 [next] 1001.8: 

Seconds : 0 from time: 1.00
HalfSecs: 0 from time: 0.50
10thS   : 0 from time: 0.11
2008.3 T1477 [next] 2007.2: 

Seconds : 1 from time: 2.01
HalfSecs: 1 from time: 1.01
10thS   : 1 from time: 0.21
3011.7 T1480 [next] 3010.6: 

Seconds : 2 from time: 3.01
HalfSecs: 2 from time: 1.51
10thS   : 2 from time: 0.32
4014.4 T1483 [next] 4013.3: 

Seconds : 3 from time: 4.01
HalfSecs: 3 from time: 2.01
10thS   : 3 from time: 0.42
5018.9 T1484 [next] 5017.8: 

Seconds : 4 from time: 5.02
HalfSecs: 4 from time: 2.52
10thS   : 4 from time: 0.52
5019.9 T1484 [cmpl] 5018.8: fin


## ... and emitting the items from only the most-recently emitted of those Observables **[switch_latest](http://reactivex.io/documentation/operators/and-then-when.html)**

In [71]:
rst(O.switch_latest)
s = O.range(0, 3).select(lambda x: O.range(x, 3)\
                                  # showing from which stream our current value comes:
                                  .map(lambda v: '%s (from stream nr %s)' % (v, x)))\
                 .switch_latest()
d = subs(s)

function switch_latest of module rx.linq.observable.switchlatest:
Transforms an observable sequence of observable sequences into an
    observable sequence producing values only from the most recent
    observable sequence.

    :returns: The observable sequence that at any point in time produces the
    elements of the most recent inner observable sequence that has been
    received.
    :rtype: Observable
    
--------------------------------------------------------------------------------

   0.9     M New subscription on stream 275998049
   1.8     M [next]    0.8: 0 (from stream nr 0)
   2.9     M [next]    1.9: 1 (from stream nr 1)
   3.5     M [next]    2.5: 2 (from stream nr 2)
   3.9     M [next]    2.9: 3 (from stream nr 2)
   4.0     M [next]    3.0: 4 (from stream nr 2)
   4.2     M [cmpl]    3.2: fin
